<i><b>Public AI</b></i>
<br>
# VGG Net 논문 읽기

### _Objective_
* VGGNet의 논문을 읽고 구현하면서, VGGNet 모델을 이해해 보겠습니다. <br>


In [35]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# \[ Paper Summary \] 
---

VGG Network는 이미지 분류 대회인 ILSVRC 대회에서 2등을 기록한 모델입니다. 구조적으로 매우 간단하면서도, 높은 성능을 기록해, 이후 많은 모델들이 이를 기반으로 구성되었습니다. <br>

![Imgur](https://i.imgur.com/l5DgFQo.png)

Reference : [Very Deep Convolutional Networks For Large-Scale Image Recognition](https://arxiv.org/pdf/1409.1556.pdf)

## 1. CNN 네트워크를 설계할때의 어려움

Convolution Layer에는 개발자, 연구자가 설정해주어야 하는 다양한 설정값들이 존재합니다.

In [4]:
from tensorflow.keras.layers import Conv2D

Conv2D?

특히 중요한 설정값(Hyper-Parameter)들로는 아래와 같이 있습니다.

* `filters`: Integer, the dimensionality of the output space
* `kernel_size`: An integer or tuple/list of 2 integers, specifying the
    height and width of the 2D convolution window.
* `strides`: An integer or tuple/list of 2 integers,
    specifying the strides of the convolution along the height and width.
* `padding`: one of `"valid"` or `"same"` (case-insensitive).
* `activation`: Activation function to use.


filters의 갯수를 얼마로 하냐, kernel_size를 얼마로 하냐 등에 따라 CNN 모델의 성능은 천차만별로 나타나게 됩니다.<br>
뿐만 아니라 Convolution Layer를 병렬로 연결할 수도 있고, 직렬로 연결할 수 있고, Pooling Layer를 어떻게 연결하냐도 우리는 고민해야하는 문제입니다
그렇기 때문에 개발자와 연구자들은 수없이 많은 실험을 통해 성공하는 모델을 도출해야 했습니다.

2014년, VGG Network가 만들어질 때 쯤 Google에서 만든 GoogleNet이라는 네트워크가 있는데, 그 구조는 아래와 같습니다.

![](https://miro.medium.com/max/5176/1*ZFPOSAted10TPd3hBQU8iQ.png)

멀리서 보기만 하더라도 매우 복잡한 구조를 가지고 있습니다. 이렇게 만들어진 네트워크는 성능이 우수했지만, 다른 개발자들이 이용하기에는 너무 모델이 복잡하고, 튜닝이 어렵다는 단점이 존재했습니다.

## 2. VGG 네트워크의 구조 

VGG 네트워크는 이와 다르게, 매우 단순하게 만들어진 모델입니다.

![Imgur](https://i.imgur.com/oANjdNh.png)

VGG Network는 Convolution Layer와 Pooling Layer을 반복적으로 쌓고, Convolutino Layer의 주요 Hyper-Parameter들을 아래와 같이 통일하였습니다.

* `kernel_size`: (3, 3)
* `strides`: (1, 1)
* `padding`: "same"
* `activation`: RELU

그리고 filters는 Pooling Layer을 지날 때마다 2배씩 증가하는 식으로 구성하였습니다. 이런 단순한 설계로도 위 GoogleNe과 거의 비슷한 수준의 성능을 내서, 이후 많은 연구자와 개발자들은 VGG Network 구조와 유사한 방식으로 모델을 설계하기 시작했습니다.

# \[ Paper Implementation \]
---

<br>

## 1. Input 구성하기

![Imgur](https://i.imgur.com/HNgH2B6.png)

* Input의 크기는 (224,224,3)입니다.<br>
* 학습시킨 데이터셋의 라벨 수는 1,000개입니다.<br>
* 전처리로서는 image의 mean value를 빼주는 식으로 진행되었습니다.<br>

In [36]:
from tensorflow.keras.layers import Input

In [37]:
input_shape = (224,224,3)
num_classes = 1000

inputs = Input(input_shape, name='images')

rgb_mean = np.array([123.68, 116.779, 103.939], np.float32)
preprocessed = #

<br>

## 2. Inference Network 구성하기

![Imgur](https://i.imgur.com/16z2FhL.png)

위의 네트워크 중 D 타입으로 구현해보도록 하겠습니다. Paper 마다 자신의 네트워크를 보다 잘 설명하기 위한 표기방법이 있습니다. VGGNet에서는 `conv<receptive field size>-<number of channels>`방식으로 크기를 설명해주었습니다.

### (1) VGG Net의 세부 사항들 

Convolution Layer의 구성은 아래 논문에 서술 되어 있습니다.

![Imgur](https://i.imgur.com/EZWrJkY.png)

* stride : 1
* padding : SAME
* Activation Function : RELU

### (2) 첫번째 VGG Block 구성하기

VGG Net은 단순히 (3,3) 크기의 Convolution Layer가 Maxpool과 함께 적층되어 있는 구조입니다.<br> 
이전의 Network들은 (5,5), (11,11) 크기의 Filter들을 이용했는데, <br>
VGG Net은 (3,3)만으로 충분히 깊게 쌓는 방식으로 모델을 구성하였습니다.

In [38]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [39]:
### Block 1
conv = #
conv = #
pool = #

그리고 많은 모델에는 Regularization을 통해 모델의 학습이 안정화합니다. <br>
해당 모델은 weight decay를 추가시켜 모델의 학습을 도왔습니다.

![Imgur](https://i.imgur.com/xVlbHUd.png)

In [40]:
from tensorflow.keras.regularizers import l2

In [41]:
### Block 1
conv = #
conv = #
pool = #

### (3) 2~5번째 VGG Block 구성하기

![Imgur](https://i.imgur.com/uTGrtng.png)

VGG 연구자들은 filter size가 (7,7)인 Convolution Layer 한 층을 구성하는 것보다<br>
filter size가 (3,3)인 Convolution Layer 3 층을 구성하는 것이<br>
파라미터 수를 줄이면서, Non-Linearlity를 확보할 수 있었다고 말합니다.

이는 현대 많은 Neural Network가 큰 Filter를 쓰지 않고, (3,3) Filter만으로 깊게 쌓도록 <br>
하는데 결정적인 연구가 되었습니다.

In [42]:
### Block 2
conv = #
conv = #
pool = #

### Block 3
conv = #
conv = #
conv = #
pool = #
    
### Block 4    
conv = #
conv = #
conv = #
pool = #
    
### Block 5
conv = #
conv = #
conv = #
pool = #

### (4) Receptive Field 계산하기

Convolution Layer을 적층하였을 때, 가장 먼저 체크해야 하는 것 중 하나가<br>
Convolution Layer의 Receptive Field 크기가 영상을 파악하기에 충분한 크기 수준으로<br>
확보되었는가입니다.

| Layer | Filter size | Stride | Receptive Field |
| ----  | ------ | ----- | ------ |
| block1-conv1 | (3,3) | (1,1) | (3,3) |
| block1-conv2 | (3,3) | (1,1) | (5,5) |
| maxpool1 | (2,2) | (2,2) | (6,6) |
| block2-conv1 | (3,3) | (1,1) | (10,10) |
| block2-conv2 | (3,3) | (1,1) | (14,14) |
| maxpool2 | (2,2) | (2,2) | (16,16) |
| block3-conv1 | (3,3) | (1,1) | (24,24) |
| block3-conv2 | (3,3) | (1,1) | (32,32) |
| block3-conv3 | (3,3) | (1,1) | (40,40) |
| maxpool3 | (2,2) | (2,2) | (44,44) |
| block4-conv1 | (3,3) | (1,1) | (60,60) |
| block4-conv2 | (3,3) | (1,1) | (76,76) |
| block4-conv3 | (3,3) | (1,1) | (92,92) |
| maxpool4 | (2,2) | (2,2) | (100,100) |
| block5-conv1 | (3,3) | (1,1) | (132,132) |
| block5-conv2 | (3,3) | (1,1) | (164,164) |
| block5-conv3 | (3,3) | (1,1) | (196,196) |
| maxpool5 | (2,2) | (2,2) | (212,212) |

마지막 Max pool layer의 1 pixel 당 (212,212)만큼의 입력값 공간을 바라보고 있습니다.<br>
영상의 크기가 (224,224)이므로 receptive Field의 크기가 충분하다고 파악할 수 있습니다.<br>

### (5) Fully Connected Layer 구성하기

![Imgur](https://i.imgur.com/jTYH9Ze.png)

논문에는 세부 구현 내용들이 곳곳히 숨겨져 있습니다.<br>
Fully Connected Layer에는 Dropout으로 Regularization을 해주어야 합니다.<br>


In [43]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [44]:
### Block 6
fc = #
fc = #

dropout = #
fc = #

dropout = #
pred = #

### (6) 모델 구성하기

In [45]:
from tensorflow.keras.models import Model

model = Model(inputs, pred)
model.summary()

<br>

## 3. 학습에 관련된 부분들 구성하기
---


![Imgur](https://i.imgur.com/C9RtVJA.png)

multinomial Logistic은 다른 말로 Softmax Function입니다. Softmax Function에 대응되는 Loss Function으로는 주로 Categorical CrossEntropy를 이용합니다.

### (1) Loss Function 구성하기

In [30]:
from tensorflow.keras.losses import #

loss = #

그리고 이전에 추가한 L2 Regularization들은 아래와 같이 확인할 수 있습니다.

In [31]:
# l2 regularization
model.losses

[<tf.Tensor 'conv1-1_2/kernel/Regularizer_3/add:0' shape=() dtype=float32>,
 <tf.Tensor 'conv1-2_2/kernel/Regularizer_3/add:0' shape=() dtype=float32>,
 <tf.Tensor 'conv2-2/kernel/Regularizer_3/add:0' shape=() dtype=float32>,
 <tf.Tensor 'conv3-1/kernel/Regularizer_3/add:0' shape=() dtype=float32>,
 <tf.Tensor 'conv3-2/kernel/Regularizer_3/add:0' shape=() dtype=float32>,
 <tf.Tensor 'conv3-3/kernel/Regularizer_3/add:0' shape=() dtype=float32>,
 <tf.Tensor 'conv4-1/kernel/Regularizer_3/add:0' shape=() dtype=float32>,
 <tf.Tensor 'conv4-2/kernel/Regularizer_3/add:0' shape=() dtype=float32>,
 <tf.Tensor 'conv4-3/kernel/Regularizer_3/add:0' shape=() dtype=float32>,
 <tf.Tensor 'conv5-1/kernel/Regularizer_3/add:0' shape=() dtype=float32>,
 <tf.Tensor 'conv5-2/kernel/Regularizer_3/add:0' shape=() dtype=float32>,
 <tf.Tensor 'conv5-3/kernel/Regularizer_3/add:0' shape=() dtype=float32>]

### (2) Optimizer 구성하기

![Imgur](https://i.imgur.com/dVyX1fX.png)

VGG 모델을 학습시킬 때에는 Optimizer로서 Momentum Optimizer을 이용하였습니다.

In [28]:
from tensorflow.keras.optimizers import SGD

#

#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.<br>
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/05/07

---